In [4]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.4 MB/s eta 0:00:00


In [15]:
import torch
from torch_geometric.nn import GCNConv

In [16]:
class GCN(torch.nn.Module):
  def __init__(self, in_channels, hidden_channels, out_channels):
      super(GCN, self).__init__()
      self.conv1 = GCNConv(in_channels, hidden_channels)
      self.conv2 = GCNConv(hidden_channels, out_channels)

  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index)
    x = torch.relu(x)
    x = self.conv2(x, edge_index)
    return x

In [9]:
from torch_geometric.datasets import Planetoid
name = 'Cora'
dataset = Planetoid(root = '/tmp/'+name, name = name)

Processing...
Done!


In [11]:
data = dataset[0]
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [17]:
model = GCN(dataset.num_node_features, 16, dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [22]:
model.train()
for epoch in range(1000):
  optimizer.zero_grad()
  out = model(data.x, data.edge_index)
  loss = torch.nn.functional.cross_entropy(out[data.train_mask], data.y[data.train_mask])
  loss.backward()
  optimizer.step()

In [23]:
model.eval()
_, pred = model(data.x, data.edge_index).max(dim = 1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7760
